In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import PIL.Image
from tensorflow.keras.preprocessing import image as kp_image
from tensorflow.keras.applications import vgg19
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from google.colab import files

# Function to load and preprocess image
def load_img(path_to_img, max_dim=512):
    img = PIL.Image.open(path_to_img)
    long = max(img.size)
    scale = max_dim / long
    img = img.resize(
        (round(img.size[0] * scale), round(img.size[1] * scale)),
        PIL.Image.Resampling.LANCZOS  # Updated for newer Pillow
    )
    img = kp_image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    return vgg19.preprocess_input(img)


# Function to deprocess image
def deprocess_img(processed_img):
    x = processed_img.copy()
    if len(x.shape) == 4:
        x = np.squeeze(x, 0)
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

# Load VGG19
def get_model():
    vgg = vgg19.VGG19(include_top=False, weights='imagenet')
    vgg.trainable = False
    style_layers = ['block1_conv1','block2_conv1','block3_conv1','block4_conv1','block5_conv1']
    content_layers = ['block5_conv2']
    outputs = [vgg.get_layer(name).output for name in style_layers + content_layers]
    return Model([vgg.input], outputs), style_layers, content_layers

# Gram matrix
def gram_matrix(input_tensor):
    channels = int(input_tensor.shape[-1])
    a = tf.reshape(input_tensor, [-1, channels])
    n = tf.shape(a)[0]
    gram = tf.matmul(a, a, transpose_a=True)
    return gram / tf.cast(n, tf.float32)

# Compute loss
def get_loss(model, loss_weights, init_image, gram_style_features, content_features):
    style_weight, content_weight = loss_weights
    model_outputs = model(init_image)
    style_output_features = model_outputs[:len(gram_style_features)]
    content_output_features = model_outputs[len(gram_style_features):]

    style_score = tf.add_n([tf.reduce_mean((gram_style_features[i] - gram_matrix(style_output_features[i]))**2)
                            for i in range(len(gram_style_features))])
    content_score = tf.add_n([
    tf.reduce_mean((content_features[i] - content_output_features[i])**2)
    for i in range(len(content_features))
])


    style_score *= style_weight
    content_score *= content_weight
    loss = style_score + content_score
    return loss

# Compute gradients
@tf.function()
def compute_grads(cfg):
    with tf.GradientTape() as tape:
        all_loss = get_loss(**cfg)
    total_loss = all_loss
    return tape.gradient(total_loss, cfg['init_image']), all_loss

# Upload content image
print("📤 Upload the Content Image")
uploaded_content = files.upload()
content_path = list(uploaded_content.keys())[0]

# Upload style image
print("📤 Now Upload the Style Image")
uploaded_style = files.upload()
style_path = list(uploaded_style.keys())[0]

# Load images
content_img = load_img(content_path)
style_img = load_img(style_path)

# Load model
model, style_layers, content_layers = get_model()
num_content_layers = len(content_layers)
num_style_layers = len(style_layers)

# Get features
def get_feature_representations(model, content_img, style_img):
    style_outputs = model(style_img)
    content_outputs = model(content_img)

    style_features = [style_layer for style_layer in style_outputs[:num_style_layers]]
    content_features = [content_layer for content_layer in content_outputs[num_style_layers:]]
    return style_features, content_features

style_features, content_features = get_feature_representations(model, content_img, style_img)
gram_style_features = [gram_matrix(style_feature) for style_feature in style_features]

# Optimize image
init_image = tf.Variable(content_img, dtype=tf.float32)
optimizer = tf.optimizers.Adam(learning_rate=5.0)

best_loss, best_img = float('inf'), None

# Configuration for training
loss_weights = (1e-2, 1e4)
cfg = {
    'model': model,
    'loss_weights': loss_weights,
    'init_image': init_image,
    'gram_style_features': gram_style_features,
    'content_features': content_features
}

# Run style transfer
import time
epochs = 10
steps_per_epoch = 100

print("🎨 Stylizing... Please wait...")
for epoch in range(epochs):
    for step in range(steps_per_epoch):
        grads, all_loss = compute_grads(cfg)
        optimizer.apply_gradients([(grads, init_image)])
        clipped = tf.clip_by_value(init_image, -128.0, 128.0)
        init_image.assign(clipped)

        if all_loss < best_loss:
            best_loss = all_loss
            best_img = deprocess_img(init_image.numpy())

# Display result
plt.imshow(best_img)
plt.axis('off')
plt.title("🎉 Neural Style Transferred Image")
plt.show()


📤 Upload the Content Image


Saving WhatsApp Image 2025-01-28 at 10.04.02_4455a78d.jpg to WhatsApp Image 2025-01-28 at 10.04.02_4455a78d (6).jpg
📤 Now Upload the Style Image


Saving WIN_20240817_17_28_26_Pro.jpg to WIN_20240817_17_28_26_Pro.jpg


/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_88']
Received: inputs=Tensor(shape=(1, 288, 512, 3))
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor_88']
Received: inputs=Tensor(shape=(1, 512, 231, 3))
  warnings.warn(msg)


🎨 Stylizing... Please wait...
